# Introduction to probabilistic piping

In this notebook, a barebones example is given on how to create a piping fragility curve using an Excel file as input for the [ProbPipingFixedWaterlevel](/reference/ProbPipingFixedWaterlevel.html) class.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from probabilistic_piping import ProbInput, ProbPipingFixedWaterlevel

## Creating input data

Probabilistic calculations expect an instance of [ProbInput](/reference/ProbInput.html). A practical way of making this [ProbInput](/reference/ProbInput.html) instance is from a specially crafted Excel sheet.

In [ ]:
data_path = Path("../../../test/data/full_test.xlsx")
df_input = pd.read_excel(data_path, sheet_name="input", index_col=0, header=0)
df_input.head(3)

The Excel data is converted to a [ProbInput](/reference/ProbInput.html) instance using the classmethod `from_dataframe()`. The [ProbInput](/reference/ProbInput.html) class has a number of attributes: `params`, `stochasts`, `charvals`, `calc_options` and `hlist`.

Depending on the type of calculation, not all options need to be set.

In [ ]:
inp_data = ProbInput.from_dataframe(df_input)

`params` are deterministic (fixed) variables

In [ ]:
inp_data.params

`stochasts` are stochastic variables

In [ ]:
inp_data.stochasts

`charvals` are characteristic values of the stochastic variables

In [ ]:
inp_data.charvals

`calc_options` are options to be used in specific probabilistic methods

In [ ]:
inp_data.calc_options

 `hlist` is a range of water levels

In [ ]:
np.array(inp_data.hlist)

## Running a fragility curve calculation

From this input data, we can now create a probabilistic piping calculation for a fixed water level. Fragility curves can be created for the submechanisms of `heave`, `uplift`, `sellmeijer` or the combination of the former as the mechanism `combi`. 

In this example, a fragility curve is calculated for the combined mechanism. Furthermore, all stochasts are assumed to be independent (`copula=None`).

In [ ]:
prob = ProbPipingFixedWaterlevel(progress=False)
_, pc = prob.fixed_waterlevel_fragilitycurve(inp_data, z_type="combi", copula=None)

The results variable contains a list of [ProbResult](/reference/ProbResult.html) instances. A [ProbResult](/reference/ProbResult.html) is created for each water level, as each water level is a separate probabilistic calculation. A [ProbResult](/reference/ProbResult.html) instance contains detailed information of such probabilistic calculation.

In [ ]:
pc.results[-1]

## Visualizing the calculated fragility curve

Using the result object, the fragility curve can be visualized using the water levels and the conditional probabilities.

In [ ]:
fig, ax = plt.subplots()
ax.plot([r.h for r in pc.results], [r.prob_cond for r in pc.results])
ax.set_xlabel("Water level [m]")
ax.set_ylabel("Failure probability");